# Introduction to Prof-Weight

This method uses white-box access to the layers of a pre-trained complex neural network model, a training dataset and a training algorithm for a simple model (presumably with much smaller number of parameters compared to the complex model like Decision Tree, Neural Nets with very few layers) and produces a new model in the simple model hypothesis class. 

The aim is to make sure that this new simple model (trained with the help of a complex model) has a higher accuracy on the training dataset than the output of the simple model training algorithm working **only** with the training dataset. 

The reasons why one would prefer a simple model is because of resource constraints or because of better interpretability offered by the simple model.

The main idea is the following: For every sample (x,y) in the training dataset, we would like to produce a sample weight w(x,y) which indicates how easy/hard the example is to learn. w(x,y) is higher if the example is easier to learn for the complex model. Prof-Weight obtains these weights as follows:

 a) Take the complex model layer L (flattened layer output). Create dataset consisting L(x),y. 
 
 b) Train a logistic classifier (we call it the probe classifier) that uses the representation L(x) and predicts y using a linear model. Let the probabilistic confidences of the probe classifier model trained only on Layer L representation be p(x,y,L).
 
 c) Repeat steps a and b for the top K layers in the complex model.
 
 d) Now take every point (x,y) in the training dataset available for the simple model to train. Let w(x,y)= (\sum_{L} p(x,y,L))/ num of layers used. These are the new sample weights.
 
 e) Use the simple model training algorithm with w(x,y) as the sample weights.
 
 Intuition: Measure of hardness or easiness of the sample may not be indicated by the final layer confidences in a highly confident complex deep neural network model in a manner that is useful for training a much simpler model. So we track how easy is a sample to predict from lower level layers's representation. So the average of the probe classifier's predictions for top K layers being high means that that sample's prediction is confident from a much lower layer indicating easiness of the sample. For a hard example, only last few layer confidences will be higher. Earlier layer representations won't have enough distinguishability.
 
We demonstrate this method where the complex model (an 18 layer Resnet) is trained on 30000 samples of the CIFAR-10 training dataset. The simple model is a Resnet with only one Resblock. The training dataset available is the rest of the 20000 samples in the standard CIFAR-10 dataset. The test dataset is the 10000 samples as in standard CIFAR-10 dataset.

Demonstrations: a) We demonstrate how to attach probes to a specific layer on a complex model stored as a tensorflow checkpoint. b) How to evaulate the flattened layer output and then use it to train a logistic probe classifier to predict y. c) Use pre-stored probe classifiers' confidences for the top K layers (we dont show probe training on all the top K layers in the notebook. However, we do demonstrate probe classifier training on one of the layers) and form sample weights d) Train the simple model using these new sample weights using the Prof-Weight Explainer Class.

Complex Model is trained on train1 (30000 samples), Simple Model is alwasy trained on train2 (20000) samples. Probe Classifiers are trained on layer representations of the complex model on train1 samples. However, probe confidences are evaluated on train2 (on which the simple model is also trained) to provide sample weights.

 
References for this method:
  1. Amit Dhurandhar, Karthikeyan Shanmugam, Ronny Luss, Peder Olsen. "Improving Simple Models with Confidence Profiles", NeurIPS 2018.


# Evaluating a Given Layer of a Tensorflow Checkpoint

In [3]:
import sys
sys.path.append("../../")
from aix360.algorithms.profwt import train_probes
from aix360.algorithms.profwt import attach_probe_checkpoint
from aix360.datasets.cifar_dataset import CIFARDataset
import json
import numpy as np
import tensorflow as tf
import os

In [9]:
#Obtain parent directory for acccessing various data files.
parent_dir = '../../aix360/models/profwt'

## Define a path for the tensorflow checkpoint of a pre-trained complex model

This complex Resnet model has been trained using the model definitions obtained from: https://github.com/tensorflow/models/tree/master/research/resnet

In [10]:
checkpoint_path = os.path.join(parent_dir, "checkpoints/train_resnetmodel_new1_799.ckpt")

## Load the Dataset on which Layer outputs need to be evaluated. 

In [17]:
dataset_obj=CIFARDataset("../../aix360/data/cifar_data")
x_train1 =dataset_obj.load_file('cifar-10-train1-image.json')
y_train1=dataset_obj.load_file('cifar-10-train1-label.json')

## Define a filename where you want layer output to be saved.

In [18]:
run=1
os.mkdir(parent_dir+'/data')
to_save_filename=parent_dir+'/data/probe_run'+str(run)+'.npy'
print(to_save_filename)

../../aix360/models/profwt/data/probe_run1.npy


## Print Names of all Layers from the model in the checkpoint

In [19]:
attach_probe_checkpoint.print_layer_labels(checkpoint_path)
tf.reset_default_graph()

[<tf.Operation 'Placeholder' type=Placeholder>, <tf.Operation 'Placeholder_1' type=Placeholder>, <tf.Operation 'init/init_conv/DW/Initializer/random_normal/shape' type=Const>, <tf.Operation 'init/init_conv/DW/Initializer/random_normal/mean' type=Const>, <tf.Operation 'init/init_conv/DW/Initializer/random_normal/stddev' type=Const>, <tf.Operation 'init/init_conv/DW/Initializer/random_normal/RandomStandardNormal' type=RandomStandardNormal>, <tf.Operation 'init/init_conv/DW/Initializer/random_normal/mul' type=Mul>, <tf.Operation 'init/init_conv/DW/Initializer/random_normal' type=Add>, <tf.Operation 'init/init_conv/DW' type=VariableV2>, <tf.Operation 'init/init_conv/DW/Assign' type=Assign>, <tf.Operation 'init/init_conv/DW/read' type=Identity>, <tf.Operation 'init/init_conv/Conv2D' type=Conv2D>, <tf.Operation 'init/Reshape/shape' type=Const>, <tf.Operation 'init/Reshape' type=Reshape>, <tf.Operation 'unit_1_0/shared_activation/init_bn/beta/Initializer/Const' type=Const>, <tf.Operation 'uni

Identify tensor names corresponding to a) Layer whose output is of interest 
b) Input layer where the model takes in  image/data sample c) Layer where model takes in the labels to fit. 

In [21]:
#Fixing a specific operation_name to define the layer output
operation_name='unit_1_1/sub_add/add:0'
# In this case the probe is intended to be after the second Resnet Block in 18 layer Resnet for CIFAR-10

input_features_name='Placeholder:0'
label_name='Placeholder_1:0'
#These two correspond to Placeholder tensors for Feature input and label

## Tapping the Layer Output, Evaluating and Storing it in a File

attach_probe_eval() function loads a tensorflow checkpoint from a path, takes these inputs: a) Layer name whose output it needs to evaluate) Placeholder Tensor name corresponding to feature input x and c) Placeholder Tensor name corresponding to Label y  d) Data samples to evaulate the layer outputs on. 

In [22]:
pr=attach_probe_checkpoint.attach_probe_eval(input_features_name,label_name,operation_name,x_train1,y_train1,checkpoint_path)
np.save(to_save_filename,pr)

Tensor("unit_1_1/sub_add/add:0", shape=(?, 32, 32, 16), dtype=float32)
Tensor("Reshape:0", shape=(?, 16384), dtype=float32)
INFO:tensorflow:Restoring parameters from ../../aix360/models/profwt/checkpoints/train_resnetmodel_new1_799.ckpt
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
probe shape (30000, 16384)


# Training a Logistic Probe Classifier based on Layer Outputs

## Load the Layer Output File for which Probe Classifier needs to be trained.

In [23]:
# In this script, samples for probe training and probe confidence evaluations are done 
# on the layer outputs obtained from the same dataset. Load the layer output values from the file.
#In general, it can be made different by supplying a new y_train2 and probe_eval_input 
y_train2=y_train1
probe_train_input=np.load(parent_dir+'/data/probe_run1.npy')
probe_eval_input=probe_train_input

## Supply Filenames to save Probe Classifier Model, Model Confidences

In [24]:
run=1
num_classes=10
to_save_pred_filename=parent_dir+"/data/probe_pred_run"+str(run)+'.npy'
to_save_probe_model_filename=parent_dir+"/data/probe_model_run"+str(run)

## Train a Probe Classifier, Evaluate it on Layer Outputs from a Dataset,
## Store the probe confidences in a File.

In [25]:
(log,pred)=train_probes.probe_train_eval(probe_train_input,y_train1,num_classes,probe_eval_input,y_train2,to_save_probe_model_filename)
np.save(to_save_pred_filename,pred)

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Start Training Probe Model.....
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190

# Simple Model Training - Unweighted on the Dataset

In [1]:
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
import os

/Users/1j1060/tensorflow3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Import the ProfWeight Explainer Class.

In [4]:
from aix360.algorithms.profwt.profwt import ProfweightExplainer
from aix360.algorithms.profwt.resnet_keras_model import resnet_v1,lr_schedule,HParams

Open the file constaining the training dataset for training the simple model. This file could be (In this example it is different) different from the dataset used for training the complex model.

In [6]:
x_train2=dataset_obj.load_file('cifar-10-train2-image.json')
y_train2=dataset_obj.load_file('cifar-10-train2-label.json')
x_test=dataset_obj.load_file('cifar-10-test-image.json')
y_test=dataset_obj.load_file('cifar-10-test-label.json')

print('x_train shape:', x_train2.shape)
print('y_train shape:', y_train2.shape)

x_train shape: (20000, 32, 32, 3)
y_train shape: (20000, 10)


Specify checkpoint to save the model after training the simple model on x_train2,y_train2 dataset.

In [15]:
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'resnet_target_model_unweighted.h5' 
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

Specify Learning Rate Schedule and all the hyper parameters for training. In this example, these are recommended setting from a popular Keras implementation of resnet models for CIFAR-10.

In [16]:
lr_scheduler = LearningRateScheduler(lr_schedule)
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),cooldown=0,patience=5,min_lr=0.5e-6)
hps = HParams(lr_scheduler=lr_scheduler,lr_reducer=lr_reducer,batch_size=128,epochs=200,checkpoint_path=filepath,num_classes=10,complexity_param=1,optimizer=Adam(lr=lr_schedule(0)))

Learning rate:  0.001


ProfWeightExplainer Class has a fit function that trains a simple model using a provided keras model that is built by calling the resnet_v1 function specified in the model file resnet_keras_model.py

In [17]:
a=ProfweightExplainer()
m=a.fit(x_train2,y_train2,x_test,y_test,resnet_v1,hps,'neural_keras')
print("Initial Simple Model Accuracy:",m[1])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 16)   64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 16)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

Learning rate:  0.001
Epoch 5/200
19968/20000 [============================>.] - ETA: 0s - loss: 1.1156 - acc: 0.6255
Epoch 00005: val_acc did not improve
20000/20000 [==============================] - 83s 4ms/step - loss: 1.1159 - acc: 0.6253 - val_loss: 1.2336 - val_acc: 0.5560
Learning rate:  0.001
Epoch 6/200
19968/20000 [============================>.] - ETA: 0s - loss: 1.0482 - acc: 0.6499
Epoch 00006: val_acc did not improve
20000/20000 [==============================] - 78s 4ms/step - loss: 1.0483 - acc: 0.6499 - val_loss: 1.3257 - val_acc: 0.5380
Learning rate:  0.001
Epoch 7/200
19968/20000 [============================>.] - ETA: 0s - loss: 0.9800 - acc: 0.6763
Epoch 00007: val_acc did not improve
20000/20000 [==============================] - 82s 4ms/step - loss: 0.9800 - acc: 0.6763 - val_loss: 1.4286 - val_acc: 0.5180
Learning rate:  0.001
Epoch 8/200
19968/20000 [============================>.] - ETA: 0s - loss: 0.9250 - acc: 0.6953
Epoch 00008: val_acc did not improve
20

19968/20000 [============================>.] - ETA: 0s - loss: 0.2293 - acc: 0.9496
Epoch 00033: val_acc did not improve
20000/20000 [==============================] - 86s 4ms/step - loss: 0.2294 - acc: 0.9496 - val_loss: 2.3893 - val_acc: 0.5100
Learning rate:  0.001
Epoch 34/200
19968/20000 [============================>.] - ETA: 0s - loss: 0.2318 - acc: 0.9478
Epoch 00034: val_acc did not improve
20000/20000 [==============================] - 91s 5ms/step - loss: 0.2319 - acc: 0.9478 - val_loss: 2.1293 - val_acc: 0.5540
Learning rate:  0.001
Epoch 35/200
19968/20000 [============================>.] - ETA: 0s - loss: 0.2225 - acc: 0.9510
Epoch 00035: val_acc did not improve
20000/20000 [==============================] - 98s 5ms/step - loss: 0.2235 - acc: 0.9507 - val_loss: 2.0747 - val_acc: 0.5560
Learning rate:  0.001
Epoch 36/200
19968/20000 [============================>.] - ETA: 0s - loss: 0.2295 - acc: 0.9485
Epoch 00036: val_acc did not improve
20000/20000 [====================

19968/20000 [============================>.] - ETA: 0s - loss: 0.0823 - acc: 0.9999
Epoch 00091: val_acc did not improve
20000/20000 [==============================] - 88s 4ms/step - loss: 0.0823 - acc: 1.0000 - val_loss: 2.2553 - val_acc: 0.6100
Learning rate:  0.0001
Epoch 92/200
19968/20000 [============================>.] - ETA: 0s - loss: 0.0815 - acc: 1.0000
Epoch 00092: val_acc did not improve
20000/20000 [==============================] - 88s 4ms/step - loss: 0.0815 - acc: 1.0000 - val_loss: 2.2517 - val_acc: 0.6160
Learning rate:  0.0001
Epoch 93/200
19968/20000 [============================>.] - ETA: 0s - loss: 0.0808 - acc: 1.0000
Epoch 00093: val_acc did not improve
20000/20000 [==============================] - 89s 4ms/step - loss: 0.0808 - acc: 1.0000 - val_loss: 2.2757 - val_acc: 0.6060
Learning rate:  0.0001
Epoch 94/200
19968/20000 [============================>.] - ETA: 0s - loss: 0.0806 - acc: 1.0000
Epoch 00094: val_acc did not improve
20000/20000 [=================

19968/20000 [============================>.] - ETA: 0s - loss: 0.0681 - acc: 1.0000
Epoch 00120: val_acc did not improve
20000/20000 [==============================] - 84s 4ms/step - loss: 0.0681 - acc: 1.0000 - val_loss: 2.5083 - val_acc: 0.5900
Learning rate:  0.0001
Epoch 121/200
19968/20000 [============================>.] - ETA: 0s - loss: 0.0672 - acc: 1.0000
Epoch 00121: val_acc did not improve
20000/20000 [==============================] - 84s 4ms/step - loss: 0.0672 - acc: 1.0000 - val_loss: 2.4561 - val_acc: 0.6000
Learning rate:  1e-05
Epoch 122/200
19968/20000 [============================>.] - ETA: 0s - loss: 0.0668 - acc: 1.0000
Epoch 00122: val_acc did not improve
20000/20000 [==============================] - 84s 4ms/step - loss: 0.0668 - acc: 1.0000 - val_loss: 2.4685 - val_acc: 0.6000
Learning rate:  1e-05
Epoch 123/200
19968/20000 [============================>.] - ETA: 0s - loss: 0.0667 - acc: 1.0000
Epoch 00123: val_acc did not improve
20000/20000 [================

19968/20000 [============================>.] - ETA: 0s - loss: 0.0646 - acc: 1.0000
Epoch 00149: val_acc did not improve
20000/20000 [==============================] - 86s 4ms/step - loss: 0.0646 - acc: 1.0000 - val_loss: 2.4768 - val_acc: 0.6000
Learning rate:  1e-05
Epoch 150/200
19968/20000 [============================>.] - ETA: 0s - loss: 0.0645 - acc: 1.0000
Epoch 00150: val_acc did not improve
20000/20000 [==============================] - 92s 5ms/step - loss: 0.0645 - acc: 1.0000 - val_loss: 2.4797 - val_acc: 0.6020
Learning rate:  1e-05
Epoch 151/200
19968/20000 [============================>.] - ETA: 0s - loss: 0.0644 - acc: 1.0000
Epoch 00151: val_acc did not improve
20000/20000 [==============================] - 87s 4ms/step - loss: 0.0644 - acc: 1.0000 - val_loss: 2.4785 - val_acc: 0.5980
Learning rate:  1e-05
Epoch 152/200
19968/20000 [============================>.] - ETA: 0s - loss: 0.0643 - acc: 1.0000
Epoch 00152: val_acc did not improve
20000/20000 [=================

19968/20000 [============================>.] - ETA: 0s - loss: 0.0633 - acc: 1.0000
Epoch 00178: val_acc did not improve
20000/20000 [==============================] - 91s 5ms/step - loss: 0.0633 - acc: 1.0000 - val_loss: 2.4904 - val_acc: 0.6040
Learning rate:  1e-06
Epoch 179/200
19968/20000 [============================>.] - ETA: 0s - loss: 0.0633 - acc: 0.9999
Epoch 00179: val_acc did not improve
20000/20000 [==============================] - 91s 5ms/step - loss: 0.0633 - acc: 1.0000 - val_loss: 2.4898 - val_acc: 0.6040
Learning rate:  1e-06
Epoch 180/200
19968/20000 [============================>.] - ETA: 0s - loss: 0.0632 - acc: 1.0000
Epoch 00180: val_acc did not improve
20000/20000 [==============================] - 91s 5ms/step - loss: 0.0632 - acc: 1.0000 - val_loss: 2.4898 - val_acc: 0.6000
Learning rate:  1e-06
Epoch 181/200
19968/20000 [============================>.] - ETA: 0s - loss: 0.0633 - acc: 1.0000
Epoch 00181: val_acc did not improve
20000/20000 [=================

# Simple Model training with Prof Weight- Sample Weights Obtained from Probe Confidences of Various Layers.

List of all filenames - each of which contains the probe confidences of a specific layer corresponding to the samples in x_train2,y_train2 dataset. This is assumed to have been obtained using functions in attach_probe_checkpoint.py and train_probes.py.

In [18]:
list_probe_filenames=[parent_dir+'/probe_output/probe_2_out_pred'+str(x)+'.npy' for x in range(10,17)]

Specify a new checkpoint for the simple model with Prof Weight + set identical hyper parameters for learning rate schedule and training.

In [19]:
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'resnet_target_model_weighted.h5' 
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)
hps = HParams(lr_scheduler=lr_scheduler,lr_reducer=lr_reducer,batch_size=128,epochs=200,checkpoint_path=filepath,num_classes=10,complexity_param=1,optimizer=Adam(lr=lr_schedule(0)))

Learning rate:  0.001


Call the ProfWeight Explainer Class's explain function - This is same as the fit function but additionally specifies list of probe filenames and start and end layer whose confidences need to be averaged to be used as the sample weights. This explain function also scores the new simple model obtained after weighted training on the test data set.

In [20]:
a.explain(x_train2,y_train2,x_test,y_test,resnet_v1,hps,list_probe_filenames,2,6,'neural_keras')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_10 (Conv2D)              (None, 32, 32, 16)   448         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_8 (BatchNor (None, 32, 32, 16)   64          conv2d_10[0][0]                  
__________________________________________________________________________________________________
activation_8 (Activation)       (None, 32, 32, 16)   0           batch_normalization_8[0][0]      
__________________________________________________________________________________________________
conv2d_11 

Learning rate:  0.001
Epoch 5/200
19968/20000 [============================>.] - ETA: 0s - loss: 0.7146 - acc: 0.6343
Epoch 00005: val_acc did not improve
20000/20000 [==============================] - 83s 4ms/step - loss: 0.7145 - acc: 0.6344 - val_loss: 1.5109 - val_acc: 0.4760
Learning rate:  0.001
Epoch 6/200
19968/20000 [============================>.] - ETA: 0s - loss: 0.6566 - acc: 0.6574
Epoch 00006: val_acc did not improve
20000/20000 [==============================] - 86s 4ms/step - loss: 0.6565 - acc: 0.6572 - val_loss: 1.6429 - val_acc: 0.5040
Learning rate:  0.001
Epoch 7/200
19968/20000 [============================>.] - ETA: 0s - loss: 0.6042 - acc: 0.6796
Epoch 00007: val_acc improved from 0.56800 to 0.59400, saving model to /Users/1j1060/Research/Codes/Transfer/Runs-neurips-2019/saved_models/resnet_target_model_weighted.h5
20000/20000 [==============================] - 84s 4ms/step - loss: 0.6045 - acc: 0.6794 - val_loss: 1.1950 - val_acc: 0.5940
Learning rate:  0.001


Learning rate:  0.001
Epoch 33/200
19968/20000 [============================>.] - ETA: 0s - loss: 0.1378 - acc: 0.8904
Epoch 00033: val_acc did not improve
20000/20000 [==============================] - 81s 4ms/step - loss: 0.1380 - acc: 0.8903 - val_loss: 2.2082 - val_acc: 0.5620
Learning rate:  0.001
Epoch 34/200
19968/20000 [============================>.] - ETA: 0s - loss: 0.1395 - acc: 0.8878
Epoch 00034: val_acc did not improve
20000/20000 [==============================] - 81s 4ms/step - loss: 0.1395 - acc: 0.8877 - val_loss: 1.9936 - val_acc: 0.6060
Learning rate:  0.001
Epoch 35/200
19968/20000 [============================>.] - ETA: 0s - loss: 0.1248 - acc: 0.8975
Epoch 00035: val_acc did not improve
20000/20000 [==============================] - 81s 4ms/step - loss: 0.1248 - acc: 0.8976 - val_loss: 2.5040 - val_acc: 0.5200
Learning rate:  0.001
Epoch 36/200
19968/20000 [============================>.] - ETA: 0s - loss: 0.1251 - acc: 0.8941
Epoch 00036: val_acc improved from 

Learning rate:  0.001
Epoch 61/200
19968/20000 [============================>.] - ETA: 0s - loss: 0.1360 - acc: 0.8936
Epoch 00061: val_acc did not improve
20000/20000 [==============================] - 83s 4ms/step - loss: 0.1361 - acc: 0.8936 - val_loss: 4.9183 - val_acc: 0.3640
Learning rate:  0.001
Epoch 62/200
19968/20000 [============================>.] - ETA: 0s - loss: 0.1366 - acc: 0.8943
Epoch 00062: val_acc did not improve
20000/20000 [==============================] - 82s 4ms/step - loss: 0.1365 - acc: 0.8943 - val_loss: 3.1165 - val_acc: 0.5560
Learning rate:  0.001
Epoch 63/200
19968/20000 [============================>.] - ETA: 0s - loss: 0.0972 - acc: 0.9142
Epoch 00063: val_acc did not improve
20000/20000 [==============================] - 82s 4ms/step - loss: 0.0972 - acc: 0.9142 - val_loss: 2.2720 - val_acc: 0.5880
Learning rate:  0.001
Epoch 64/200
19968/20000 [============================>.] - ETA: 0s - loss: 0.0856 - acc: 0.9199
Epoch 00064: val_acc improved from 

Learning rate:  0.0001
Epoch 88/200
19968/20000 [============================>.] - ETA: 0s - loss: 0.0658 - acc: 0.9344
Epoch 00088: val_acc did not improve
20000/20000 [==============================] - 80s 4ms/step - loss: 0.0658 - acc: 0.9344 - val_loss: 1.7468 - val_acc: 0.6780
Learning rate:  0.0001
Epoch 89/200
19968/20000 [============================>.] - ETA: 0s - loss: 0.0655 - acc: 0.9343
Epoch 00089: val_acc did not improve
20000/20000 [==============================] - 80s 4ms/step - loss: 0.0656 - acc: 0.9342 - val_loss: 1.7672 - val_acc: 0.6760
Learning rate:  0.0001
Epoch 90/200
19968/20000 [============================>.] - ETA: 0s - loss: 0.0651 - acc: 0.9341
Epoch 00090: val_acc did not improve
20000/20000 [==============================] - 80s 4ms/step - loss: 0.0651 - acc: 0.9342 - val_loss: 1.7668 - val_acc: 0.6820
Learning rate:  0.0001
Epoch 91/200
19968/20000 [============================>.] - ETA: 0s - loss: 0.0648 - acc: 0.9357
Epoch 00091: val_acc improved f

Learning rate:  1e-05
Epoch 145/200
19968/20000 [============================>.] - ETA: 0s - loss: 0.0487 - acc: 0.9404
Epoch 00145: val_acc did not improve
20000/20000 [==============================] - 86s 4ms/step - loss: 0.0487 - acc: 0.9403 - val_loss: 1.9060 - val_acc: 0.6840
Learning rate:  1e-05
Epoch 146/200
19968/20000 [============================>.] - ETA: 0s - loss: 0.0487 - acc: 0.9407
Epoch 00146: val_acc did not improve
20000/20000 [==============================] - 86s 4ms/step - loss: 0.0487 - acc: 0.9407 - val_loss: 1.8993 - val_acc: 0.6780
Learning rate:  1e-05
Epoch 147/200
19968/20000 [============================>.] - ETA: 0s - loss: 0.0486 - acc: 0.9405
Epoch 00147: val_acc did not improve
20000/20000 [==============================] - 86s 4ms/step - loss: 0.0486 - acc: 0.9405 - val_loss: 1.9037 - val_acc: 0.6820
Learning rate:  1e-05
Epoch 148/200
19968/20000 [============================>.] - ETA: 0s - loss: 0.0485 - acc: 0.9410
Epoch 00148: val_acc did not im

Learning rate:  1e-06
Epoch 174/200
19968/20000 [============================>.] - ETA: 0s - loss: 0.0472 - acc: 0.9414
Epoch 00174: val_acc did not improve
20000/20000 [==============================] - 86s 4ms/step - loss: 0.0472 - acc: 0.9415 - val_loss: 1.9188 - val_acc: 0.6880
Learning rate:  1e-06
Epoch 175/200
19968/20000 [============================>.] - ETA: 0s - loss: 0.0471 - acc: 0.9411
Epoch 00175: val_acc did not improve
20000/20000 [==============================] - 87s 4ms/step - loss: 0.0471 - acc: 0.9410 - val_loss: 1.9202 - val_acc: 0.6880
Learning rate:  1e-06
Epoch 176/200
19968/20000 [============================>.] - ETA: 0s - loss: 0.0471 - acc: 0.9408
Epoch 00176: val_acc did not improve
20000/20000 [==============================] - 86s 4ms/step - loss: 0.0471 - acc: 0.9408 - val_loss: 1.9205 - val_acc: 0.6880
Learning rate:  1e-06
Epoch 177/200
19968/20000 [============================>.] - ETA: 0s - loss: 0.0472 - acc: 0.9418
Epoch 00177: val_acc did not im